##Домашняя работа № 2 Коллаборативная фильтрация

##Датасет MovieLens 1M

In [2]:
import pandas as pd
import numpy as np

from tqdm import tqdm

In [3]:
# загрузим данные
!wget 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'

--2025-01-29 10:59:55--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  2.31MB/s    in 0.4s    

2025-01-29 10:59:55 (2.31 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]



In [4]:
# распакуем
import zipfile

with zipfile.ZipFile('/content/ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall('unpacked_files')

In [5]:
# прочитаем
links = pd.read_csv('/content/unpacked_files/ml-latest-small/links.csv')
movies = pd.read_csv('/content/unpacked_files/ml-latest-small/movies.csv')
ratings = pd.read_csv('/content/unpacked_files/ml-latest-small/ratings.csv')
tags = pd.read_csv('/content/unpacked_files/ml-latest-small/tags.csv')

In [6]:
# посмотрим
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [7]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [9]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


##Surprise

https://surpriselib.com/

Алгоритмы https://surprise.readthedocs.io/en/stable/prediction_algorithms_package.html

In [46]:
# установим
!pip install surprise

In [167]:
from surprise import KNNWithMeans, KNNBasic, KNNWithZScore, KNNBaseline
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise import Reader
from surprise.model_selection import train_test_split

Соберем и исследуем наш датасет

In [48]:
# соберем наш датасет
movies_with_ratings = movies.merge(ratings, on='movieId').reset_index(drop=True)
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [49]:
# проверим на пропуски
movies_with_ratings.isna().sum()

,0
movieId,0
title,0
genres,0
userId,0
rating,0
timestamp,0


In [50]:
# проверим на нулевые значения
movies_with_ratings.isnull().sum()

,0
movieId,0
title,0
genres,0
userId,0
rating,0
timestamp,0


In [51]:
# посмотрим на наши рейтинги
movies_with_ratings['rating'].value_counts().sort_values()

,count
rating,
0.5,1370
1.5,1791
1.0,2811
2.5,5550
2.0,7551
4.5,8551
3.5,13136
5.0,13211
3.0,20047


In [52]:
# соберем наш датасет
dataset = pd.DataFrame({'uid': movies_with_ratings.userId,
                        'iid': movies_with_ratings.title,
                        'rating': movies_with_ratings.rating
})
dataset.head(1)

,uid,iid,rating
0,1,Toy Story (1995),4.0


In [53]:
# еще раз посмотрим на рейтинги
ratings.rating.min(), ratings.rating.max()

(0.5, 5.0)

In [54]:
# сформируем объект surprise
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [55]:
data

In [56]:
# разобъем на выборки
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [57]:
# посмотрим, что это
trainset, testset, type(testset)

(<surprise.trainset.Trainset at 0x78818badc350>,
 [(217, 'Fly Away Home (1996)', 4.0),
  (184, 'Escape Plan (2013)', 4.0),
  (263, 'Secretary (2002)', 4.5),
  (414, 'Dark Blue World (Tmavomodrý svet) (2001)', 4.0),
  (380, '50 First Dates (2004)', 3.0),
  (76, 'Star Wars: Episode I - The Phantom Menace (1999)', 2.0),
  (608, 'Cruel Intentions (1999)', 3.0),
  (79, 'Godfather, The (1972)', 5.0),
  (290, 'Simple Plan, A (1998)', 4.0),
  (599, 'Safety of Objects, The (2001)', 1.5),
  (322, 'My Cousin Vinny (1992)', 3.5),
  (226, 'Sleepy Hollow (1999)', 2.5),
  (465, 'Who Framed Roger Rabbit? (1988)', 4.0),
  (305, 'War of the Worlds (2005)', 3.5),
  (390, 'Children of a Lesser God (1986)', 4.5),
  (59, 'Witness (1985)', 5.0),
  (420, 'Monty Python Live at the Hollywood Bowl (1982)', 4.0),
  (464, 'Sudden Death (1995)', 2.5),
  (167, 'Pulp Fiction (1994)', 3.0),
  (332, 'Dune (1984)', 1.5),
  (401, 'Goofy Movie, A (1995)', 4.5),
  (263, 'Bend It Like Beckham (2002)', 4.0),
  (177, 'Tallade

In [58]:
# посмотрим на количество
dataset['uid'].nunique(), dataset['iid'].nunique()

(610, 9719)

##KNNWithMeans (default)

In [171]:
# построим модель по умолчанию
algo = KNNWithMeans()
algo.fit(trainset)

# проверим точность
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8922


0.8922136195806034

In [170]:
# сделаем кросс-валидацию на всем датасете
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8990  0.9009  0.8956  0.8955  0.8978  0.8978  0.0021  
MAE (testset)     0.6862  0.6882  0.6859  0.6846  0.6855  0.6861  0.0012  
Fit time          0.12    0.15    0.14    0.14    0.14    0.14    0.01    
Test time         1.21    1.15    1.17    1.17    1.79    1.30    0.25    


{'test_rmse': array([0.89900665, 0.90094763, 0.89556058, 0.89546672, 0.8978265 ]),
 'test_mae': array([0.68618165, 0.6882044 , 0.68591278, 0.68462745, 0.68546006]),
 'fit_time': (0.12028861045837402,
  0.14659380912780762,
  0.13719964027404785,
  0.1395721435546875,
  0.13968133926391602),
 'test_time': (1.2073938846588135,
  1.1546180248260498,
  1.1690001487731934,
  1.165395736694336,
  1.786043643951416)}

##KNNBasic (default)

In [172]:
# построим модель по умолчанию
algo = KNNBasic()
algo.fit(trainset)

# проверим точность
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9413


0.9413274667769663

In [173]:
# сделаем кросс-валидацию на всем датасете
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9503  0.9439  0.9477  0.9418  0.9561  0.9480  0.0050  
MAE (testset)     0.7295  0.7224  0.7244  0.7233  0.7335  0.7266  0.0042  
Fit time          0.08    0.11    0.11    0.11    0.11    0.10    0.01    
Test time         1.06    1.10    1.08    1.04    1.06    1.07    0.02    


{'test_rmse': array([0.95029155, 0.94388221, 0.94768346, 0.9417722 , 0.95613301]),
 'test_mae': array([0.72947948, 0.7224366 , 0.7243794 , 0.72334657, 0.73352775]),
 'fit_time': (0.08014583587646484,
  0.10881519317626953,
  0.10603070259094238,
  0.1126260757446289,
  0.1116020679473877),
 'test_time': (1.0614311695098877,
  1.0991485118865967,
  1.0843064785003662,
  1.0413129329681396,
  1.058570146560669)}

##KNNWithZScore (default)

In [174]:
# построим модель по умолчанию
algo = KNNWithZScore()
algo.fit(trainset)

# проверим точность
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8924


0.8924197320059594

In [175]:
# сделаем кросс-валидацию на всем датасете
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithZScore on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8966  0.8917  0.8928  0.9005  0.8990  0.8961  0.0034  
MAE (testset)     0.6815  0.6768  0.6757  0.6862  0.6778  0.6796  0.0038  
Fit time          0.17    0.25    0.20    0.21    0.20    0.21    0.03    
Test time         1.98    1.21    1.26    1.29    1.24    1.40    0.29    


{'test_rmse': array([0.89656824, 0.89170284, 0.89277086, 0.90050046, 0.89903592]),
 'test_mae': array([0.68145921, 0.67682026, 0.675722  , 0.68623692, 0.67780526]),
 'fit_time': (0.17125773429870605,
  0.25492143630981445,
  0.19832229614257812,
  0.21318364143371582,
  0.1988201141357422),
 'test_time': (1.9754807949066162,
  1.2124762535095215,
  1.258638858795166,
  1.2942957878112793,
  1.2357630729675293)}

##KNNBaseline

In [102]:
# построим модель по умолчанию
algo = KNNBaseline()
algo.fit(trainset)

# проверим точность
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8706


0.870583574696182

In [126]:
# сделаем кросс-валидацию на всем датасете
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8681  0.8854  0.8701  0.8718  0.8761  0.8743  0.0062  
MAE (testset)     0.6649  0.6734  0.6670  0.6664  0.6697  0.6683  0.0030  
Fit time          0.47    0.42    0.42    0.45    0.43    0.44    0.02    
Test time         2.14    1.28    1.33    1.34    1.36    1.49    0.33    


{'test_rmse': array([0.8681132 , 0.88544407, 0.87010877, 0.87176608, 0.87614211]),
 'test_mae': array([0.66489312, 0.67335247, 0.6670072 , 0.6664306 , 0.66965212]),
 'fit_time': (0.471027135848999,
  0.4181504249572754,
  0.41903138160705566,
  0.45104408264160156,
  0.43422436714172363),
 'test_time': (2.144437551498413,
  1.2783279418945312,
  1.334825038909912,
  1.3398082256317139,
  1.356358528137207)}

Уже интереснее! Исследуем подробнее

In [176]:
from surprise.model_selection import GridSearchCV

In [163]:
# зададим сетку параметров
param_grid = {#'bsl_options': {'method': ['als', 'sgd']},
              'bsl_options': {'method': ['als']},
                              #'reg': [1, 2],
                              'reg': [1],
              #'k': [40, 50, 60],
              #'k': [50],
              #'sim_options': {'name': ['msd', 'cosine', 'pearson', 'pearson_baseline'],
              'sim_options': {'name': ['pearson_baseline'],
                              #'min_support': [1, 3, 5],
                              'min_support': [1],
                              #'user_based': [True, False]}
                              'user_based': [False]}
}

# Инициализация GridSearchCV
gs = GridSearchCV(KNNBaseline, param_grid, measures=['rmse'], cv=5)
gs.fit(data)

# выведем лучшие параметры
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
0.852239332220442
{'bsl_options': {'method': 'als'}, 'reg': 1, 'sim_options': {'name': 'pearson_baseline', 'min_support': 1, 'user_based': False}}


In [164]:
# построим и обучим нащу лучшую модель на тестовой выборке
best_param = gs.best_params['rmse']
algo = KNNBaseline(params=best_param)
algo.fit(trainset)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [165]:
# проверим точность
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8706


0.870583574696182

In [166]:
# сделаем кросс-валидацию на всем датасете
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8750  0.8779  0.8746  0.8711  0.8720  0.8741  0.0024  
MAE (testset)     0.6679  0.6682  0.6686  0.6685  0.6661  0.6679  0.0009  
Fit time          0.42    0.42    0.41    0.45    0.47    0.43    0.02    
Test time         1.34    1.32    1.34    1.32    2.34    1.53    0.40    


{'test_rmse': array([0.87503804, 0.87791743, 0.87462187, 0.87108989, 0.87204427]),
 'test_mae': array([0.66794309, 0.66815991, 0.66860499, 0.66845708, 0.66614396]),
 'fit_time': (0.41524314880371094,
  0.42046260833740234,
  0.41430139541625977,
  0.44940662384033203,
  0.47116732597351074),
 'test_time': (1.3353633880615234,
  1.3236174583435059,
  1.335777997970581,
  1.320462703704834,
  2.339890480041504)}

Точность на вализации модели по умолчанию смог улучшить на 0.0002. Чисто символически

На тестовой выборки лучший показатель был 0.85. Но на валидации показали были хуже